In [9]:
import requests

import pandas as pd
import numpy as np

import joblib

# Чтобы сгенерировать UUID
import uuid


In [10]:
UUID = uuid.uuid4().hex

In [11]:
UUID

'f9cad96f21534f1ab1eaa6f4d417d852'

In [12]:
URL_BEGIN_DATA = 'https://lab.karpov.courses/hardml-api/project-1/task/{uuid}/begin'
req = requests.post(URL_BEGIN_DATA.format(uuid=UUID))

# Для получения информации о том как отработал запрос, можно посмотреть ответ
print(req.json())

{'status': 'ok'}


In [13]:
#df

## Inference Time

In [14]:
URL_BEGIN_DATA = 'https://lab.karpov.courses/hardml-api/project-1/task/{uuid}/begin'
URL_TASK_DATA_GET = 'https://lab.karpov.courses/hardml-api/project-1/task/{uuid}/data'
URL_TASK_RESULT_POST = 'https://lab.karpov.courses/hardml-api/project-1/task/{uuid}/result/'

def plug_date_features(df, date_colname):
    df[date_colname] = pd.to_datetime(df[date_colname])
    df['year'] = df[date_colname].dt.year
    df['month'] = df[date_colname].dt.month
    df['week_num'] = df[date_colname].dt.isocalendar().week.astype(pd.Int64Dtype())# wasn't present in score=5 solution
    df['day'] = df[date_colname].dt.day
    return df

model = joblib.load('Подход Урока 4/model.j')

features_besides_transaction = pd.read_csv('features_besides_transaction_raw.csv')

In [20]:
response = requests.get(URL_TASK_DATA_GET.format(uuid=UUID))

response_json = response.json()
print(f'response: {response_json}') #<Response [200]>

# Чтобы перевести полученный ответ в pd.DataFrame
df = pd.read_json(response_json)

sub_df = df[['dates', 'SKU']]
sub_df = plug_date_features(sub_df, 'dates')

df['price'] = model.predict(sub_df.drop(columns='dates').values)

req = requests.post(URL_TASK_RESULT_POST.format(uuid=UUID),
                         data=df.to_json(orient='records'))
print(f'req: {req.json()}')

URL_TASK_RESULT_GET = 'https://lab.karpov.courses/hardml-api/project-1/task/{uuid}/result'
response = requests.get(URL_TASK_RESULT_GET.format(uuid=UUID))

# Чтобы перевести полученный ответ в pd.DataFrame
df = pd.read_json(response.json())

df.head()

response: <Response [200]>


ValueError: Invalid file path or buffer object type: <class 'dict'>

In [ ]:
response = requests.get(URL_TASK_DATA_GET.format(uuid=UUID))

response_json = response.json()
print(f'response: {response_json}') #<Response [200]>

# Чтобы перевести полученный ответ в pd.DataFrame
df = pd.read_json(response_json)

sub_df = df[['dates', 'SKU']]
sub_df = plug_date_features(sub_df, 'dates')

sub_df = sub_df.merge(features_besides_transaction, on=['SKU', 'year', 'month', 'week_num', 'day'], how='left')

df['price'] = model.predict(sub_df.drop(columns='dates').values)

req = requests.post(URL_TASK_RESULT_POST.format(uuid=UUID),
                         data=df.to_json(orient='records'))
print(f'req: {req.json()}')

URL_TASK_RESULT_GET = 'https://lab.karpov.courses/hardml-api/project-1/task/{uuid}/result'
response = requests.get(URL_TASK_RESULT_GET.format(uuid=UUID))

# Чтобы перевести полученный ответ в pd.DataFrame
df = pd.read_json(response.json())

df.head()

# Data Ingestion

In [21]:
canc_df = pd.read_csv('data/canc_df.csv')
promo_df = pd.read_csv('data/promo_df.csv')
sales_plan = pd.read_csv('data/sales_plan.csv')
transactions = pd.read_csv('data/transactions.csv')

In [22]:
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334260 entries, 0 to 1334259
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   dates   1334260 non-null  object 
 1   SKU     1334260 non-null  int64  
 2   user    1334260 non-null  int64  
 3   price   1334260 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 40.7+ MB


,dates,SKU,user,price
0,2018-11-09,16019,44796,1252.0
1,2018-11-09,16019,44510,1252.0
2,2018-11-09,16019,19074,1252.0
3,2018-11-09,16019,34472,1252.0
4,2018-11-10,16019,48407,1252.0


In [346]:
canc_df.info()
canc_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10324 entries, 0 to 10323
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             10324 non-null  int64  
 1   week_num         10324 non-null  int64  
 2   SKU              10324 non-null  int64  
 3   ret_net_1_price  10324 non-null  float64
 4   ret_net_2_price  10324 non-null  float64
 5   ret_net_3_price  10324 non-null  float64
dtypes: float64(3), int64(3)
memory usage: 484.1 KB


,year,week_num,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price
0,2019,1,10121,2945.681342,2965.743797,2917.394830
1,2019,1,10290,949.466060,942.301292,962.799009
2,2019,1,10973,3751.515022,3735.814981,3709.586540
3,2019,1,11179,3717.042306,3638.799082,3695.090121
4,2019,1,11680,1378.797669,1325.251312,1363.707489


In [347]:
promo_df.info()
promo_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2342 entries, 0 to 2341
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SKU       2342 non-null   int64  
 1   week_num  2342 non-null   int64  
 2   year      2342 non-null   int64  
 3   discount  2342 non-null   float64
dtypes: float64(1), int64(3)
memory usage: 73.3 KB


,SKU,week_num,year,discount
0,74430,1,2018,0.15
1,69978,1,2018,0.05
2,81955,1,2018,0.15
3,78953,1,2018,0.10
4,43034,1,2018,0.05


In [348]:
sales_plan.info()
sales_plan.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12212 entries, 0 to 12211
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SKU         12212 non-null  int64  
 1   year        12212 non-null  int64  
 2   month       12212 non-null  int64  
 3   plan        12212 non-null  float64
 4   back_bonus  12212 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 477.2 KB


,SKU,year,month,plan,back_bonus
0,10290,2018,2,20.0,120000
1,10290,2018,3,60.0,140000
2,10290,2018,4,70.0,140000
3,10290,2018,5,60.0,140000
4,10290,2018,6,60.0,40000


# Joining Tables

In [23]:
transactions.dates = pd.to_datetime(transactions.dates)
transactions.sort_values(by=['dates', 'SKU'], inplace=True)
transactions = transactions.groupby(['dates', 'SKU']).price.agg(np.median).reset_index()
#transactions.SKU = pd.Categorical(transactions.SKU)
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318956 entries, 0 to 318955
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   dates   318956 non-null  datetime64[ns]
 1   SKU     318956 non-null  int64         
 2   price   318956 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 7.3 MB


,dates,SKU,price
0,2018-01-01,67369,3607.0
1,2018-01-02,67369,3607.0
2,2018-01-03,26629,1126.0
3,2018-01-03,41262,3473.0
4,2018-01-03,67369,3607.0


In [24]:
transactions.describe()

,SKU,price
count,318956.000000,318956.000000
mean,53078.770736,2312.815523
std,25955.136878,1245.596328
min,10060.000000,101.000000
25%,29179.000000,1252.000000
50%,52820.000000,2197.000000
75%,74550.000000,3465.000000
max,99996.000000,4649.000000


In [25]:
def plug_date_features(df, date_colname):
    df[date_colname] = pd.to_datetime(df[date_colname])
    df['year'] = df[date_colname].dt.year
    df['month'] = df[date_colname].dt.month
    df['week_num'] = df[date_colname].dt.isocalendar().week.astype(pd.Int64Dtype())
    df['day'] = df[date_colname].dt.day
    return df

In [26]:
transactions = plug_date_features(transactions, 'dates')
transactions.head()

,dates,SKU,price,year,month,week_num,day
0,2018-01-01,67369,3607.0,2018,1,1,1
1,2018-01-02,67369,3607.0,2018,1,1,2
2,2018-01-03,26629,1126.0,2018,1,1,3
3,2018-01-03,41262,3473.0,2018,1,1,3
4,2018-01-03,67369,3607.0,2018,1,1,3


In [27]:
transactions.tail()

,dates,SKU,price,year,month,week_num,day
318951,2019-11-30,98842,1368.0,2019,11,48,30
318952,2019-11-30,99266,1891.0,2019,11,48,30
318953,2019-11-30,99310,448.0,2019,11,48,30
318954,2019-11-30,99974,3307.0,2019,11,48,30
318955,2019-11-30,99996,813.0,2019,11,48,30


In [28]:
def make_dates_int(df):
    for col in ['year', 'month', 'week_num']:
        if col in df.columns:
            print(f'Processing {col}')
            df[col].fillna(pd.NA)
            df[col] = df[col].astype(pd.Int64Dtype())
#     df.month = df.month.astype(pd.Int64Dtype())
#     df.week_num = df.week_num.astype(pd.Int64Dtype())
    return df

## Putting together features_besides_transaction, p.1

In [355]:
def get_inds(params: dict, df: pd.DataFrame):
    inds = np.repeat([True], len(df))
    for param in params:
        if param in df.columns:
            new_inds = df[param] == params[param]
            print(f"{param} detected: {new_inds.sum()} records satisfy")
            inds = inds & new_inds
        else:
            print(f"{param} NOT detected")
    return inds

In [356]:
all_dates = pd.date_range(start='2018-01-01', end='2019-12-31')
all_dates_df = pd.DataFrame({'dates': all_dates})
all_dates_df = plug_date_features(all_dates_df, 'dates')
all_dates_df.drop(columns=['dates'], inplace=True)
all_dates_df = make_dates_int(all_dates_df)
all_dates_df.info()
all_dates_df

Processing year
Processing month
Processing week_num
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   year      730 non-null    Int64
 1   month     730 non-null    Int64
 2   week_num  730 non-null    Int64
 3   day       730 non-null    int64
dtypes: Int64(3), int64(1)
memory usage: 25.1 KB


,year,month,week_num,day
0,2018,1,1,1
1,2018,1,1,2
2,2018,1,1,3
3,2018,1,1,4
4,2018,1,1,5
...,...,...,...,...
725,2019,12,52,27
726,2019,12,52,28
727,2019,12,52,29
728,2019,12,1,30


In [357]:
inds = get_inds({'year': 2018, 'month': 1, 'SKU': 67369}, all_dates_df)
all_dates_df[inds]

year detected: 365 records satisfy
month detected: 62 records satisfy
SKU NOT detected


,year,month,week_num,day
0,2018,1,1,1
1,2018,1,1,2
2,2018,1,1,3
3,2018,1,1,4
4,2018,1,1,5
5,2018,1,1,6
6,2018,1,1,7
7,2018,1,2,8
8,2018,1,2,9
9,2018,1,2,10


In [358]:
# features_besides_transaction = all_dates_df.merge(canc_df, on=['year', 'week_num'], how='outer')
# features_besides_transaction.fillna(-99, inplace=True)
# features_besides_transaction = make_dates_int(features_besides_transaction)
# features_besides_transaction

canc_df_all_dates = all_dates_df.merge(canc_df, on=['year', 'week_num'], how='outer')
canc_df_all_dates.fillna(-99, inplace=True)
canc_df_all_dates = make_dates_int(canc_df_all_dates)
canc_df_all_dates

Processing year
Processing month
Processing week_num


,year,month,week_num,day,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price
0,2018,1,1,1,-99.0,-99.000000,-99.000000,-99.000000
1,2018,1,1,2,-99.0,-99.000000,-99.000000,-99.000000
2,2018,1,1,3,-99.0,-99.000000,-99.000000,-99.000000
3,2018,1,1,4,-99.0,-99.000000,-99.000000,-99.000000
4,2018,1,1,5,-99.0,-99.000000,-99.000000,-99.000000
...,...,...,...,...,...,...,...,...
72882,2019,12,52,29,97659.0,3409.219608,3394.456244,3434.336572
72883,2019,12,52,29,98346.0,1632.598644,1710.327024,1638.673052
72884,2019,12,52,29,98642.0,1498.441016,1520.235649,1481.583880
72885,2019,12,52,29,98774.0,1649.812737,1638.274072,1725.167364


In [359]:
inds = get_inds({'year': 2018, 'month': 1, 'SKU': 67369}, canc_df_all_dates)
canc_df_all_dates[inds]

year detected: 365 records satisfy
month detected: 5132 records satisfy
SKU detected: 0 records satisfy


,year,month,week_num,day,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price


In [360]:
# features_besides_transaction = features_besides_transaction.merge(promo_df, on=['year', 'week_num', 'SKU'], how='outer')
# features_besides_transaction.fillna(-99, inplace=True)
# features_besides_transaction = make_dates_int(features_besides_transaction)

promo_df_all_dates = all_dates_df.merge(promo_df, on=['year', 'week_num'], how='outer')
promo_df_all_dates.fillna(-99, inplace=True)
promo_df_all_dates = make_dates_int(promo_df_all_dates)
promo_df_all_dates

Processing year
Processing month
Processing week_num


,year,month,week_num,day,SKU,discount
0,2018,1,1,1,74430,0.15
1,2018,1,1,1,69978,0.05
2,2018,1,1,1,81955,0.15
3,2018,1,1,1,78953,0.10
4,2018,1,1,1,43034,0.05
...,...,...,...,...,...,...
16428,2019,12,52,29,35645,0.15
16429,2019,12,52,29,81314,0.05
16430,2019,12,52,29,75661,0.05
16431,2019,12,52,29,38664,0.05


In [361]:
inds = get_inds({'year': 2018, 'month': 1, 'SKU': 67369}, promo_df_all_dates)
promo_df_all_dates[inds]

year detected: 4409 records satisfy
month detected: 909 records satisfy
SKU detected: 0 records satisfy


,year,month,week_num,day,SKU,discount


In [362]:
# features_besides_transaction = features_besides_transaction.merge(sales_plan, on=['year', 'month', 'SKU'], how='outer')
# features_besides_transaction.fillna(-99, inplace=True)
# features_besides_transaction = make_dates_int(features_besides_transaction)

sales_plan_all_dates = all_dates_df.merge(sales_plan, on=['year', 'month'], how='outer')
sales_plan_all_dates.fillna(-99, inplace=True)
sales_plan_all_dates = make_dates_int(sales_plan_all_dates)
sales_plan_all_dates

Processing year
Processing month
Processing week_num


,year,month,week_num,day,SKU,plan,back_bonus
0,2018,1,1,1,11318,0.0,160000
1,2018,1,1,1,11582,0.0,160000
2,2018,1,1,1,11666,20.0,100000
3,2018,1,1,1,14640,50.0,100000
4,2018,1,1,1,14749,10.0,140000
...,...,...,...,...,...,...,...
372135,2019,12,1,31,98842,60.0,140000
372136,2019,12,1,31,99266,150.0,120000
372137,2019,12,1,31,99310,70.0,140000
372138,2019,12,1,31,99974,30.0,60000


In [363]:
inds = get_inds({'year': 2018, 'month': 1, 'SKU': 67369}, sales_plan_all_dates)
sales_plan_all_dates[inds]

year detected: 104539 records satisfy
month detected: 18755 records satisfy
SKU detected: 730 records satisfy


,year,month,week_num,day,SKU,plan,back_bonus
30,2018,1,1,1,67369,210.0,120000
76,2018,1,1,2,67369,210.0,120000
122,2018,1,1,3,67369,210.0,120000
168,2018,1,1,4,67369,210.0,120000
214,2018,1,1,5,67369,210.0,120000
260,2018,1,1,6,67369,210.0,120000
306,2018,1,1,7,67369,210.0,120000
352,2018,1,2,8,67369,210.0,120000
398,2018,1,2,9,67369,210.0,120000
444,2018,1,2,10,67369,210.0,120000


## Putting together features_besides_transaction, p.2

In [364]:
features_besides_transaction = canc_df_all_dates.merge(promo_df_all_dates, on=['year','month','week_num','day','SKU'], how='outer')
features_besides_transaction.fillna(-99, inplace=True)
features_besides_transaction = make_dates_int(features_besides_transaction)
features_besides_transaction

Processing year
Processing month
Processing week_num


,year,month,week_num,day,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount
0,2018,1,1,1,-99.0,-99.0,-99.0,-99.0,-99.00
1,2018,1,1,2,-99.0,-99.0,-99.0,-99.0,-99.00
2,2018,1,1,3,-99.0,-99.0,-99.0,-99.0,-99.00
3,2018,1,1,4,-99.0,-99.0,-99.0,-99.0,-99.00
4,2018,1,1,5,-99.0,-99.0,-99.0,-99.0,-99.00
...,...,...,...,...,...,...,...,...,...
88438,2019,12,52,29,86769.0,-99.0,-99.0,-99.0,0.05
88439,2019,12,52,29,35645.0,-99.0,-99.0,-99.0,0.15
88440,2019,12,52,29,81314.0,-99.0,-99.0,-99.0,0.05
88441,2019,12,52,29,75661.0,-99.0,-99.0,-99.0,0.05


In [365]:
inds = get_inds({'year': 2018, 'month': 1, 'SKU': 67369}, features_besides_transaction)
features_besides_transaction[inds]

year detected: 4774 records satisfy
month detected: 6004 records satisfy
SKU detected: 0 records satisfy


,year,month,week_num,day,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount


In [366]:
features_besides_transaction = features_besides_transaction.merge(sales_plan_all_dates, on=['year','month','week_num','day','SKU'], how='outer')
features_besides_transaction.fillna(-99, inplace=True)
features_besides_transaction = make_dates_int(features_besides_transaction)
features_besides_transaction

Processing year
Processing month
Processing week_num


,year,month,week_num,day,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount,plan,back_bonus
0,2018,1,1,1,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1,2018,1,1,2,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
2,2018,1,1,3,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
3,2018,1,1,4,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
4,2018,1,1,5,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...
373304,2019,12,1,31,98842.0,-99.0,-99.0,-99.0,-99.0,60.0,140000.0
373305,2019,12,1,31,99266.0,-99.0,-99.0,-99.0,-99.0,150.0,120000.0
373306,2019,12,1,31,99310.0,-99.0,-99.0,-99.0,-99.0,70.0,140000.0
373307,2019,12,1,31,99974.0,-99.0,-99.0,-99.0,-99.0,30.0,60000.0


In [367]:
inds = get_inds({'year': 2018, 'month': 1, 'SKU': 67369}, features_besides_transaction)
features_besides_transaction[inds]

year detected: 104952 records satisfy
month detected: 19516 records satisfy
SKU detected: 730 records satisfy


,year,month,week_num,day,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount,plan,back_bonus
88473,2018,1,1,1,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88519,2018,1,1,2,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88565,2018,1,1,3,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88611,2018,1,1,4,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88657,2018,1,1,5,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88703,2018,1,1,6,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88749,2018,1,1,7,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88795,2018,1,2,8,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88840,2018,1,2,9,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0
88885,2018,1,2,10,67369.0,-99.0,-99.0,-99.0,-99.0,210.0,120000.0


In [37]:
features_besides_transaction = features_besides_transaction[features_besides_transaction.SKU != -99]
features_besides_transaction.SKU = features_besides_transaction.SKU.astype(int)

In [38]:
# add  categorical encoding
# missing values?
features_besides_transaction.to_csv('features_besides_transaction_raw.csv', index=False)

In [341]:
def print_all_info_for_params(params: dict, dfs: []):
    for i, df in enumerate(dfs):
        print(f"DF #{i}")
        inds = get_inds(params, df)
        print(f"===Total Inds: {inds.sum()}")
        print(df[inds], '\n')

In [369]:
params = {'year': 2018, 'month': 1, 'week_num': 2, 'SKU': 67369}
print_all_info_for_params(params, [features_besides_transaction, transactions])

DF #0
year detected: 104952 records satisfy
month detected: 19516 records satisfy
week_num detected: 4242 records satisfy
SKU detected: 730 records satisfy
===Total Inds: 7
       year  month  week_num  day      SKU  ret_net_1_price  ret_net_2_price  \
88795  2018      1         2    8  67369.0            -99.0            -99.0   
88840  2018      1         2    9  67369.0            -99.0            -99.0   
88885  2018      1         2   10  67369.0            -99.0            -99.0   
88930  2018      1         2   11  67369.0            -99.0            -99.0   
88975  2018      1         2   12  67369.0            -99.0            -99.0   
89020  2018      1         2   13  67369.0            -99.0            -99.0   
89065  2018      1         2   14  67369.0            -99.0            -99.0   

       ret_net_3_price  discount   plan  back_bonus  
88795            -99.0     -99.0  210.0    120000.0  
88840            -99.0     -99.0  210.0    120000.0  
88885            -99.0  

# Combining transactions with features

In [50]:
features_besides_transaction = pd.read_csv('features_besides_transaction_raw.csv')
features_besides_transaction.head()

,year,month,week_num,day,SKU,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount,plan,back_bonus
0,2019,1,1,1,10121,2945.681342,2965.743797,2917.394830,-99.0,-99.0,-99.0
1,2019,1,1,1,10290,949.466060,942.301292,962.799009,-99.0,70.0,40000.0
2,2019,1,1,1,10973,3751.515022,3735.814981,3709.586540,-99.0,100.0,140000.0
3,2019,1,1,1,11179,3717.042306,3638.799082,3695.090121,-99.0,90.0,40000.0
4,2019,1,1,1,11680,1378.797669,1325.251312,1363.707489,-99.0,-99.0,-99.0


In [55]:
transactions = pd.read_csv('data/transactions.csv')
transactions.dates = pd.to_datetime(transactions.dates)
transactions.sort_values(by=['dates', 'SKU'], inplace=True)
transactions = transactions.groupby(['dates', 'SKU']).price.agg(np.median).reset_index()
#transactions.SKU = pd.Categorical(transactions.SKU)
transactions = plug_date_features(transactions, 'dates')
transactions.info()
transactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318956 entries, 0 to 318955
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   dates     318956 non-null  datetime64[ns]
 1   SKU       318956 non-null  int64         
 2   price     318956 non-null  float64       
 3   year      318956 non-null  int64         
 4   month     318956 non-null  int64         
 5   week_num  318956 non-null  Int64         
 6   day       318956 non-null  int64         
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(4)
memory usage: 17.3 MB


,dates,SKU,price,year,month,week_num,day
0,2018-01-01,67369,3607.0,2018,1,1,1
1,2018-01-02,67369,3607.0,2018,1,1,2
2,2018-01-03,26629,1126.0,2018,1,1,3
3,2018-01-03,41262,3473.0,2018,1,1,3
4,2018-01-03,67369,3607.0,2018,1,1,3


In [56]:
combo_df = transactions.merge(features_besides_transaction, on=['SKU', 'year', 'month', 'week_num', 'day'], how='left')
combo_df

,dates,SKU,price,year,month,week_num,day,ret_net_1_price,ret_net_2_price,ret_net_3_price,discount,plan,back_bonus
0,2018-01-01,67369,3607.0,2018,1,1,1,-99.000000,-99.000000,-99.000000,-99.0,210.0,120000.0
1,2018-01-02,67369,3607.0,2018,1,1,2,-99.000000,-99.000000,-99.000000,-99.0,210.0,120000.0
2,2018-01-03,26629,1126.0,2018,1,1,3,-99.000000,-99.000000,-99.000000,-99.0,140.0,180000.0
3,2018-01-03,41262,3473.0,2018,1,1,3,-99.000000,-99.000000,-99.000000,-99.0,170.0,100000.0
4,2018-01-03,67369,3607.0,2018,1,1,3,-99.000000,-99.000000,-99.000000,-99.0,210.0,120000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
318951,2019-11-30,98842,1368.0,2019,11,48,30,-99.000000,-99.000000,-99.000000,-99.0,60.0,60000.0
318952,2019-11-30,99266,1891.0,2019,11,48,30,-99.000000,-99.000000,-99.000000,-99.0,150.0,80000.0
318953,2019-11-30,99310,448.0,2019,11,48,30,-99.000000,-99.000000,-99.000000,-99.0,30.0,120000.0
318954,2019-11-30,99974,3307.0,2019,11,48,30,3309.093239,3318.713486,3313.977814,-99.0,30.0,100000.0


In [58]:
combo_df.to_csv('combo_df_raw.csv', index=False)